<a href="https://colab.research.google.com/github/noambassat/SupremeCourtClassifier/blob/main/7_BOTH_MODELES_PREDICTIONS_add_CONTRYSIDE_columns.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import re
import pandas as pd
import matplotlib.pyplot as plt
from pandas.api.types import CategoricalDtype
import ast
import joblib
from transformers import AutoTokenizer, AutoModel
import numpy as np

In [ ]:
full_ra_rap = pd.read_excel("full_ra_rap.xlsx")

In [ ]:
full_rap = full_ra_rap[full_ra_rap["סוג הליך"]=='רע"פ']
# בדיקה אם בעמודת "גוף המסמך" קיימת המילה "רע"פ"
full_rap = full_rap[full_rap["גוף המסמך"].str.contains('רע"פ', na=False)]

In [ ]:
full_rap

In [ ]:
full_rap.shape

In [ ]:
full_rap[["גוף המסמך"]].dropna(how='any', ignore_index=True,inplace=True)
full_rap[["מספר הליך","שם הליך"]].dropna(how='all', ignore_index=True,inplace=True)
full_rap[["מספר הליך","שם הליך"]].drop_duplicates(inplace=True, ignore_index=True)

In [ ]:
# פונקציה להמרת טקסט שמופיע כרשימה למחרוזת רגילה
def convert_list_to_string(text):
    if isinstance(text, str) and text.startswith("[") and text.endswith("]"):
        try:
            # מנסה להמיר את התוכן בתוך הסוגריים לרשימה אמיתית
            text_list = ast.literal_eval(text)
            # איחוד המחרוזות לרצף טקסט אחד
            return ' '.join(text_list)
        except (ValueError, SyntaxError):
            return text
    return text

# פונקציה לניקוי התווים המיותרים
def clean_text(text):
    if isinstance(text, list):
        text = ' '.join(text)  # הפיכת רשימה למחרוזת
    elif isinstance(text, str):
        # הסרת תווי רווח מיותרים ותווים מיוחדים
        text = re.sub(r'\n+', ' ', text)  # הסרת שורות חדשות מרובות
        text = re.sub(r'\\n', '', text)   # הסרת תווי newline \n מהטקסט
        text = re.sub(r'\\xa0', ' ', text)  # הסרת תווי \xa0 מהטקסט
        text = re.sub(r'\s+', ' ', text)  # הסרת רווחים מרובים
        return text.strip()
    return text

# הדפסת שורות לא קריאות לפני המרה
print("שורות לא קריאות לפני המרה:")
print(full_rap["גוף המסמך"].head())

# המרה של הטקסטים הלא קריאים לטקסטים קריאים ושמירה על הדאטה המקורי
full_rap["גוף המסמך"] = full_rap["גוף המסמך"].apply(clean_text)

print("\nשורות לאחר המרה:")
print(full_rap["גוף המסמך"].head())

file_path = 'full_rap_df_cleaned.csv'
full_rap.to_csv(file_path, index=False, encoding='utf-8')

# print(f"הקובץ נשמר בהצלחה בנתיב: {file_path}")


In [ ]:
# בדיקה אם הערכים השתנו
changed_rows = full_rap["גוף המסמך"] != full_rap["גוף המסמך"].apply(clean_text)
print(f"מספר שורות ששונו: {changed_rows.sum()}")
print("שורות ששונו:")
print(full_rap.loc[changed_rows, "גוף המסמך"].head())


In [ ]:
full_rap

In [ ]:
# הגדרת פונקציה לעיצוב טקסט
def format_text(text, line_length=80):
    """
    פורמט טקסט כך שיהיה נוח לקריאה עם שורות שאורכן מוגבל.
    """
    import textwrap
    return "\n".join(textwrap.wrap(text, width=line_length))


In [ ]:
# ביטויים לבדיקה
keywords = ["העותק כפוף לשינויי עריכה וניסוח", "העתק מתאים", 'ניתנההיום', 'ניתןהיום', 'ניתן היום', 'ניתנה היום',
            "העתקמתאים", "מזכיר ראשי", "supreme.court.gov.il", "מרכז מידע", "מרכזמידע"]

# בדיקה אילו שורות מכילות אחד מהביטויים
rows_with_keyword = full_rap["גוף המסמך"].apply(
    lambda x: any(keyword in x for keyword in keywords) if isinstance(x, str) else False
)

# סינון שורות שלא מכילות את הביטוי
filtered_data = full_rap[rows_with_keyword]

# הדפסת מספר השורות שנמחקו
num_removed = len(full_rap) - len(filtered_data)  # **שונה לחישוב מדויק**
print(f"\nמספר השורות שנמחקו: {num_removed}")

# **שינוי לצורך סעיף 6**: בדיקה אם נותרו ערכים חסרים בעמודת "גוף המסמך"
missing_values_count = filtered_data["גוף המסמך"].isna().sum()
if missing_values_count > 0:
    print(f"\nנותרו {missing_values_count} ערכים חסרים בעמודת 'גוף המסמך' לאחר סינון.")
else:
    print("\nאין ערכים חסרים בעמודת 'גוף המסמך' לאחר סינון.")

# שמירת הנתונים המסוננים לקובץ חדש
filtered_data.to_csv("full_rap_df_cleaned.csv", index=False, encoding='utf-8')
print("\nהנתונים המסוננים נשמרו לקובץ: 'new_filtered_data.csv'")


full_rap = filtered_data.copy()  # **שונה להעתקה בטוחה**


In [ ]:
full_rap

In [ ]:
for col in full_rap.columns:
  print(col)

In [ ]:
sides = full_rap[[
"צד א'",
"צד ב'"
]]

In [ ]:
values = [val for value in sides.values for val in value]
values

In [ ]:
clean_values = set()
for val in values:
    val_str = str(val)  # המרה למחרוזת
    cleaned_val = re.sub(r"[\[\]'\\n]", "", val_str).strip()
    clean_values.add(cleaned_val)

In [ ]:
clean_values

In [ ]:
def check_sub(value,subs):
  for sub in subs:
    if(value.find(sub)!=-1):
      return 1
  return 0

In [ ]:
private_sides = set()
public_sides = set()
for value in clean_values:
  if(check_sub(value,["רשות מקרקעי ישראל","מדינת ","מועצה ","שירות ה","היחידה הארצי","לאכיפת ","הרשות ל","המשרד ל",'יועמ"ש',"המשפטי לממשלה","מנהל מקרקעי","פרקליטות","שירות בתי הסוהר",'שב"ס',"מינהל מקרקעי","עירית ","רשויות","איגוד ערים","עיריית ","מדינת ישראל","משרד ה","רשות ה","אלוף ",'יו"ר',"יושב ראש","מדינת ישראל","ראש עיריית","ועדה","צבא","מפקד","פיקוד","משטרת"])==0):
    private_sides.add(value)
  else:
    public_sides.add(value)

In [ ]:
for side in private_sides:
  print(side)

In [ ]:
for side in public_sides:
  print(side)

In [ ]:
sides_A = full_rap[[
"צד א'"
]]

values_A = [val for value in sides_A.values for val in value]
values_A

In [ ]:
def check_contry_side(value):
    # רשימת מילות מפתח
    subs = [
        "רשות מקרקעי ישראל", "מדינת ", "מועצה ", "שירות ה", "היחידה הארצי", "לאכיפת ", "הרשות ל",
        "המשרד ל", 'יועמ"ש', "המשפטי לממשלה", "מנהל מקרקעי", "פרקליטות", "שירות בתי הסוהר", 'שב"ס',
        "מינהל מקרקעי", "עירית ", "רשויות", "איגוד ערים", "עיריית ", "מדינת ישראל", "משרד ה", "מקרקעי ישראל",
        "רשות ה", "אלוף ", 'יו"ר', "יושב ראש", "ראש עיריית", "ועדה", "צבא", "מפקד", "פיקוד", "משטרת"
    ]

    # ניקוי רווחים והפיכת המחרוזת לאותיות קטנות
    try:
      value = value.strip()

    except AttributeError:
      return 0

    # בדיקה אם אחת המחרוזות קיימת ב-value
    if any(sub in value for sub in subs):
        return 1
    print(value)

    return 0

In [ ]:
check_none = full_rap[full_rap["צד א'"].isna()]
check_none.loc[:,"צד ב'"]

In [ ]:
full_rap[full_rap["צד א'"].isna() & full_rap["צד ב'"].isna()].shape[0]

In [ ]:
full_rap.dropna(subset=["צד א'","צד ב'"],how='all',inplace=True)

In [ ]:
full_rap["מדינת ישראל בצד א'"] = full_rap["צד א'"].apply(lambda x: check_contry_side(x))

# Create checked data - to remove from predictions

In [ ]:
dca = pd.read_csv("dca_for_classifier.csv")

In [ ]:
ignore_df_1 = pd.read_excel("full_data_BOTH_models_with_predictions_V.xlsx")
ignore_df_1.dropna(subset=['בקשה לרשות ערעור התקבלה?'],inplace=True)
ignore_df_2 = pd.read_excel("2_full_data_with_predictions V2.xlsx")
ignore_df_2.dropna(subset=['בקשה לרשות ערעור התקבלה?'],inplace=True)
ignore_df_3 = pd.read_excel("full_data_with_predictions-VETTING.xlsx")
ignore_df_3.dropna(subset=['בקשה לרשות ערעור התקבלה?'],inplace=True)
ignore_df = pd.concat([ignore_df_1,ignore_df_2,ignore_df_3,dca])
ignore_df.dropna(subset=['בקשה לרשות ערעור התקבלה?'],inplace=True)
ignore_df.drop_duplicates(subset=['מספר הליך'],keep='first',inplace=True)



In [ ]:
ignore_df.shape

In [ ]:
ignore_df['רע"פ בקשה אחרת או דלמטה מיוחד'].value_counts()

In [ ]:
ignore_df[~ignore_df['רע"פ בקשה אחרת או דלמטה מיוחד'].isna()].shape

In [ ]:
# ignore_df[(ignore_df['רע"פ בקשה אחרת או דלמטה מיוחד'] != 'בקשה אחרת') & (ignore_df['רע"פ בקשה אחרת או דלמטה מיוחד'] != 'דלמטה מיוחד')].shape

In [ ]:
ignore_df[~ignore_df['רע"פ בקשה אחרת או דלמטה מיוחד'].isna()].shape

In [ ]:
delete_from_main_df = ignore_df[~ignore_df['רע"פ בקשה אחרת או דלמטה מיוחד'].isna()]
delete_from_main_df.shape

In [ ]:
ignore_df[(ignore_df['בקשה לרשות ערעור התקבלה?']== 'אחר')].shape

In [ ]:
ignore_df.to_csv("allready_checked_data_with_dca.csv", index=False, encoding='utf-8')

In [ ]:
delete_from_main_df = pd.concat([delete_from_main_df,ignore_df[(ignore_df['בקשה לרשות ערעור התקבלה?'] == 'אחר')]])

In [ ]:
delete_from_main_df.shape

In [ ]:
delete_from_main_df.drop_duplicates(subset=['מספר הליך'],keep='first',inplace=True)

In [ ]:
delete_from_main_df.shape

In [ ]:
full_rap.shape

In [ ]:
full_rap = full_rap[~full_rap["מספר הליך"].isin(delete_from_main_df["מספר הליך"])]

In [ ]:
full_rap

# Cut doc's body for classifiers

In [ ]:

# פונקציה לחיתוך הטקסט בהתאם לכללים
def trim_text(text,min_length=500,last_sen=30):


    # שמירת אורך מקורי לאבחון
    original_length = len(text)

    # הסרת מספר תווים מסוף הטקסט
    text = text[:-last_sen]

    # חיתוך ל-min_length האחרונים אם הטקסט ארוך יותר מהמינימום
    if len(text) > min_length:
        text = text[-min_length:]

    # הדפסת פידבק רק אם הטקסט עבר שינוי
    # if len(text) != original_length:
    #     print(f"Trimmed Text (Original Length: {original_length}, Trimmed Length: {len(text)}):")
    #     formatted_text = "\n".join([text[i:i+80] for i in range(0, len(text), 80)])
    #     print(formatted_text)
    #     print("##*******************************************************####")
    return text


In [ ]:
# חיתוך הטקסטים בעמודת "גוף המסמך"
full_rap["גוף המסמך חתוך"] = full_rap["גוף המסמך"].apply(trim_text)
full_rap["גוף המסמך חתוך למסווג השני"] = full_rap["גוף המסמך"].apply(lambda x: trim_text(x, 550, 80))
# שמירת האינדקס המקורי
full_rap.reset_index(inplace=True, drop=False)


In [ ]:
full_rap.to_excel("full_rap_df_cleaned.xlsx", index=False, encoding='utf-8')

# Embeddings

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("dean-ai/sentence_transformer_Legal-heBERT")
model = AutoModel.from_pretrained("dean-ai/sentence_transformer_Legal-heBERT")

In [ ]:
def get_embeddings(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1).detach().numpy()
    return embeddings[0]

In [ ]:
# חישוב ושמירת אימבדינגים לעמודה הראשונה

print("מתחילים לחשב אימבדינגים על גוף המסמך החתוך...")
full_rap["גוף המסמך חתוך - embeddings"] = full_rap["גוף המסמך חתוך"].apply(lambda text: get_embeddings(text))
print("אימבדינגים חושבו בהצלחה!")


# First classifier predictions

In [ ]:
def model_prediction(data, name,treshold, classifier, embeddings):

  predicted_probabilities = classifier.predict_proba(embeddings)[:, 1]

  data[f"{name}_predicted_probability"] = predicted_probabilities

  data[f"{name}_original_prediction"] = classifier.predict(embeddings)

  data[f"{name}_treshold_prediction"] = (predicted_probabilities > treshold).astype(int)



  # PRINTS
  class_distribution = data[f"{name}_original_prediction"].value_counts()
  print("\nכמות Class 0 ו-Class 1 (לפי המודל המקורי):")
  print(class_distribution)

  # בדיקת התפלגות נורמלית
  normalized_distribution = data[f"{name}_original_prediction"].value_counts(normalize=True)
  print("\nהתפלגות נורמלית של Class 0 ו-Class 1 (לפי המודל המקורי):")
  print(normalized_distribution)


  # בדיקת התפלגות התחזיות עם הסף החדש
  adjusted_class_distribution = data[f"{name}_treshold_prediction"].value_counts()
  print("\nכמות Class 0 ו-Class 1 (לפי הסף המותאם):")
  print(adjusted_class_distribution)

  # בדיקת התפלגות נורמלית עם הסף החדש
  adjusted_normalized_distribution = data[f"{name}_treshold_prediction"].value_counts(normalize=True)
  print("\nהתפלגות נורמלית של Class 0 ו-Class 1 (לפי הסף המותאם):")
  print(adjusted_normalized_distribution)

  # שמירת הנתונים לקובץ Excel
  output_file_adjusted = "Full_RAAP_with_predictions.xlsx"
  data.to_excel(output_file_adjusted, index=False, engine="openpyxl")

In [ ]:
model_prediction(full_rap,"first_classifier",0.67, joblib.load("3_first_classifier_model_Triple_trained.pkl"), np.array(full_rap["גוף המסמך חתוך - embeddings"].tolist()))

# Second classifier predictions

In [ ]:
data = full_rap[full_rap["first_classifier_treshold_prediction"]==1].copy()

In [ ]:
# חישוב ושמירת אימבדינגים לעמודה השנייה
print("מתחילים לחשב אימבדינגים על גוף המסמך החתוך השני...")
data["גוף המסמך חתוך למסווג השני - embeddings"] = data["גוף המסמך חתוך למסווג השני"].apply(lambda text: get_embeddings(text))
print("אימבדינגים 2 חושבו בהצלחה!")


In [ ]:
model_prediction(data, "second_classifier",0.9, joblib.load("second_classifier_model.pkl"), np.array(data["גוף המסמך חתוך למסווג השני - embeddings"] .tolist()))

In [ ]:
full_rap = pd.concat([data,full_rap])
full_rap.drop_duplicates(subset = ["מספר הליך"],keep = 'first', inplace=True)


In [ ]:
from google.colab import drive

# חיבור Google Drive
drive.mount('/content/drive')

# הגדרת נתיב לשמירה בדרייב
file_path = '/content/drive/My Drive/full_rap_df_cleaned.xlsx'

# שמירת הקובץ
full_rap.to_excel(file_path, index=False)

print(f"File saved successfully to {file_path}")


# Remove checked data from the prediction's sample!!!

In [ ]:
full_rap.shape

In [ ]:
none_checked_raap = full_rap[~full_rap["מספר הליך"].isin(ignore_df["מספר הליך"])]

In [ ]:
none_checked_raap.shape

In [ ]:
sample_df = none_checked_raap[["מספר תיק", "מספר הליך","Year","מדינת ישראל בצד א'",
    'גוף המסמך חתוך', 'גוף המסמך חתוך למסווג השני',
       'גוף המסמך חתוך - embeddings',
       'גוף המסמך חתוך למסווג השני - embeddings',
       'first_classifier_predicted_probability',
       'first_classifier_original_prediction',
       'first_classifier_treshold_prediction',
       'second_classifier_predicted_probability',
       'second_classifier_original_prediction',
       'second_classifier_treshold_prediction']
]

In [ ]:
sample_df.shape

In [ ]:
# Extract the set of case numbers in dca
dca_cases = set(dca["מספר תיק"])
ignore_cases = set(ignore_df["מספר תיק"])
# Filter sample_df to exclude rows where 'מספר התיק' is in dca_cases
filtered_sample_df = sample_df[~sample_df["מספר תיק"].isin(dca_cases)]

filtered_sample_df = filtered_sample_df[~filtered_sample_df["מספר הליך"].isin(ignore_cases)]

In [ ]:
sample_df = filtered_sample_df.copy()
sample_df.shape

In [ ]:
sample_df = sample_df[sample_df["Year"]>=2000]

In [ ]:
len(sample_df["Year"].unique())

In [ ]:
def summarize_predictions_by_year(col):
    # Group by Year and first_classifier_treshold_prediction, then count occurrences
    summary = sample_df.groupby(["Year", col]).size().unstack(fill_value=0)
    summary.columns = ["Class 0", "Class 1"]  # Rename columns for clarity
    return summary


In [ ]:

# Usage example:
summary_df_1 = summarize_predictions_by_year("first_classifier_treshold_prediction")
summary_df_1


In [ ]:

# Usage example:
summary_df_2 = summarize_predictions_by_year("second_classifier_treshold_prediction")
summary_df_2


In [ ]:
def balance_data_with_year_2000(data, prediction_column, year_column):
    balanced_data = []
    for year, group in data.groupby(year_column):
        if year == 2000:
            # מוסיפים את כל הנתונים של שנת 2000
            balanced_data.append(group)
        else:
            # מאוזנים עבור שנים אחרות
            ones = group[group[prediction_column] == 1]
            zeros = group[group[prediction_column] == 0].sample(len(ones), random_state=42)
            balanced_data.append(pd.concat([ones, zeros]))
    # איחוד כל השנים למאגר אחד
    return pd.concat(balanced_data)

# שימוש בפונקציה
balanced_sample_df = balance_data_with_year_2000(sample_df, "first_classifier_treshold_prediction", "Year")


In [ ]:
balanced_sample_df

In [ ]:
balanced_sample_df['בקשה לרשות ערעור התקבלה?'] = None
balanced_sample_df['הערעור התקבל?'] = None
balanced_sample_df['רע"פ בקשה אחרת או דלמטה מיוחד'] = None

In [ ]:
balanced_sample_df.to_excel("final_sample_df.xlsx", index=False)